<a href="https://colab.research.google.com/github/lakshitgosain/Transformers-for-NLP/blob/main/TF_I01_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Text processing Review

Steps:
1. Tokenizing- String split, split words,+ punctuations, characters, subwords
2. Map tokens to integers . E.g I like cats= [420, 650, 103]
3. Padding/truncation (to process batches of different length)


Tokenization
* strtok function - splits the sting by a deliminator
* It's but now more than just splitting by a Deliminator. We need more steps . Eg - Lemmatization, stemming, stopwords removal

Character-level Tokenization
- BOB likes cats -> B,O,B,"",l,i,k,e,s...
- Such are used for use cases like : language translation, name generation.

Subword Token
It is customary to use subword tokeniztion in Transformers.
* sometimes we split words into multiple tokens.
* consider- run, running, etc.
* with word-level tokenization, these would be treated independently.
* Not ideal , because we know there's some relationship.
* Better with 2 tokens- run +suffix





##Subword tokenization and trnasformers

* Although we ofter use subword tokeniztion with transformers, they are separate concepts
* If you are using RNNs, we can also use subword tokenization with RNN.

Subword tokenizatio- How do we choose subword boundaries

We can apply algos to tokenize texts



###NOTE: Different models use different tokeization techniques

##Mapping tokens to integers

We represent a mapping with dict

We also use the reverse-mapping

## Padding

Every doc has a different length which is not good for processing in batchs..so we need a constant size.

We cannot assume that it's always 0

** We do not want to pad everything to be as long as the longest document in the whole dataset( There can be outliers
* EG- Most if the docs have 10k words but one doc had 100k words/tokens
* We should pad dynamically, relative to the current batch
* It's ok for different batches to have different lengths.
* Transformers cannot be recurrent, so they cannot handle sequences of different lengths


## Truncation

Opposite of padding.
* We want our docs to be shorter.
There is a limit of characters for transformers